### 1.3 sklearn 활용 (한글 뉴스 분류)

In [1]:
!wget https://github.com/kyungsoo-fininsight/mulcam_b/raw/master/data/2019news_1000.csv

--2020-07-27 00:13:07--  https://github.com/kyungsoo-fininsight/mulcam_b/raw/master/data/2019news_1000.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kyungsoo-fininsight/mulcam_b/master/data/2019news_1000.csv [following]
--2020-07-27 00:13:07--  https://raw.githubusercontent.com/kyungsoo-fininsight/mulcam_b/master/data/2019news_1000.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11125708 (11M) [text/plain]
Saving to: ‘2019news_1000.csv’

2019news_1000.csv   100%[===================>]  10.61M  19.2MB/s    in 0.6s    

2020-07-27 00:13:07 (19.2 MB/s) - ‘2019news_1000.csv’ saved [11125708/1112

In [2]:
%%time
import pandas as pd
df = pd.read_csv("./2019news_1000.csv")

CPU times: user 135 ms, sys: 31 ms, total: 166 ms
Wall time: 177 ms


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [5]:
df.head() # 컨텐츠하고 카테고리만 사용할거야!

,url,category1,category2,date,title,media,content
0,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,모바일,2019-05-02,"인권단체 중국, 신장위구르 소수민족 감시용 모바일앱 가동",연합뉴스,"HRW ""개인 정보 수집·보고서 작성·조사 활동에 앱 활용"" ""36가지 감시유형…뒷..."
1,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,모바일,2019-05-09,"카카오, 1분기 매출 7000억 넘어서…8분기째 최고치 경신(종합)",뉴시스,영업익 166.0%↑·순이익 19.
2,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,과학 일반,2019-12-11,"테라젠이텍스, '유전체 정보 관리 시스템' 특허 취득",연합뉴스,(서울=연합뉴스) 김잔디 기자 = 테라젠이텍스는 유전체 분석 정보 관리 시스템에 관...
3,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,컴퓨터,2019-10-14,"두나무-삼성증권-딥서치, 비상장 주식 통합 거래 지원 플랫폼 출범",디지털데일리,'증권플러스 비상장' 서비스를 설명중인 두나무 이성현 핀테크사업실장 [디지털데일리 ...
4,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,통신/뉴미디어,2019-10-24,"과기정통부, 태풍 미탁 피해, 특별재난지역 전파사용료 6개월간 전액감면",전자신문,과기정통부 로고 과학기술정보통신부는 18호 태풍 '미탁'으로 인해 특별재난지역으로 ...


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['category1'], random_state = 0)

text_clf = Pipeline([
                     ('vect', CountVectorizer(ngram_range=(1,2))),
                     ('tfidf', TfidfTransformer()),
                     ('clf' , MultinomialNB(alpha = 0.5))
                  
])

text_clf = text_clf.fit(X_train, y_train)

In [9]:
import numpy as np
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.6832

In [11]:
# 이제 GridSearchCV

from sklearn.model_selection import GridSearchCV
parameters_clf = {
    "vect__ngram_range" : [(1,1), (1,2), (1,3), (1,4)],
    "tfidf__use_idf" : (True, False),
    "clf__alpha" : (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)
}

gs_clf = GridSearchCV(text_clf, parameters_clf, n_jobs = -1, verbose = 2)
gs_clf = gs_clf.fit(X_train, y_train)



Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 21.6min finished


AttributeError: ignored

In [14]:
best_parameters= gs_clf.best_estimator_.get_params()

for param_name in sorted(list(best_parameters.keys())):
    print("\t{}:{}".format(param_name, best_parameters[param_name]))

predicted = gs_clf.best_estimator_.predict(X_test) 
np.mean(predicted == y_test) # 정확도 비교를 우해 ~~

	clf:MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
	clf__alpha:0.01
	clf__class_prior:None
	clf__fit_prior:True
	memory:None
	steps:[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))]
	tfidf:TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)
	tfidf__norm:l2
	tfidf__smooth_idf:True
	tfidf__sublinear_tf:False
	tfidf__use_idf:True
	vect:CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                

0.7408